In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd

In [2]:
# read bitcoin conversion rates
df_btc = pd.read_csv("BTC_EUR_conversion.csv", delimiter=",")
df_btc.Date = pd.to_datetime(df_btc['Date'])
df_btc.Price = df_btc.Price.str.replace(",", "")
df_btc.Date = df_btc.Date.dt.date
df_btc.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-12-31,6397.4,"6,449.8","6,511.5","6,335.3",3.03K,-0.81%
1,2019-12-30,6449.8,"6,606.7","6,606.7","6,430.1",3.56K,-2.37%
2,2019-12-29,6606.7,"6,536.7","6,729.8","6,519.4",2.11K,1.07%
3,2019-12-28,6536.6,"6,485.0","6,584.8","6,473.2",1.69K,0.80%
4,2019-12-27,6485.0,"6,489.9","6,507.3","6,352.0",2.84K,-0.08%


In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql
df_pw = pd.read_csv("pw.csv", delimiter=",")
db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  from ipykernel import kernelapp as app


In [4]:
query = """SELECT price_eur,
`time` 
FROM DarknetScrape.vendor_reviews
where market = 'Berlusconi Market'
and positive = 1;
"""
# """SELECT
# 	reviews.price_eur as price_eur_review,
# 	items.price_eur as price_eur_items,
# 	items.price_btc ,
# 	reviews.`time`,
# 	vendors.vendor_id ,
# 	reviews.item_id
# FROM
# 	DarknetScrape.item_reviews as reviews
# JOIN DarknetScrape.items as items on
# 	reviews.item_id = items.item_id
# join DarknetScrape.vendors vendors on
# 	vendors.vendor_id = items.vendor_id
# join DarknetScrape.market market on
# 	market.market_name = vendors.market
# WHERE
# 	reviews.positive = 1
# 	and market.market_name = "Berlusconi Market"
# """

df = pd.read_sql(query, con=db_connection)
df.head()

,price_eur,time
0,12.03,2019-07-23 19:30:31
1,12.19,2019-07-23 07:47:21
2,12.22,2019-07-21 19:14:31
3,7.33,2019-07-21 19:13:00
4,13.29,2019-07-19 07:07:59


In [5]:
# convert the euro price from the reviews into bitcoin (value at the time)
df["price_btc_review"] = "0"
for i, row in df.iterrows():
    ifor_val = str(row.price_eur / float(df_btc[df_btc["Date"] == row.time.date()]["Price"].iloc[0]))
    df.at[i,'price_btc_review'] = ifor_val
df.head()

,price_eur,time,price_btc_review
0,12.03,2019-07-23 19:30:31,0.0013592145254047701
1,12.19,2019-07-23 07:47:21,0.001377292191578067
2,12.22,2019-07-21 19:14:31,0.001293272232746669
3,7.33,2019-07-21 19:13:00,0.0007757516747981247
4,13.29,2019-07-19 07:07:59,0.001414552111716621


In [6]:
df.to_csv("reviews_price_btc.csv")